# Assignment #1: Building an inverted index
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program that collects all the words from a set of documents
* Build an index from the words
* Represent a document using the Tf.Idf values
* Write a short report of 1 to 2 pages on the assignment
* Read a description of an industrial system and answer a question on it

## Submission

When you have written all the missing code and run all the cells, you will show your notebook to one of the assistants to register that you have passed.

In case, we do not have enough assistants, you will possibly submit your notebook to an automatic marking system. More information on this later if we need it. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Description of the assignment

### Outline

In this lab, you will build an indexer to index all the words in a corpus. Conceptually, an index consists of rows with one word per row and the list of files and positions, where this word occurs. Such a row is called a _posting list_. You will encode the position of a word by the number of characters from the start of the file.
<pre>
word1: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
word2: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
...
</pre>

#### Imports

Some imports. Add others as needed

In [1]:
import bz2
import math
import os
import pickle
import regex as re
import requests
import sys
import time
from zipfile import ZipFile

## Corpus

You will create an index for a corpus of Selma Lagerlöf's works: To gather the corpus, you can alternatively:
1. Download the <a href="https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip">Selma folder</a> and uncompress it. It contains novels by <a href="https://sv.wikipedia.org/wiki/Selma_Lagerl%C3%B6f">Selma Lagerlöf</a>. The text of these novels was extracted from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at <a href="https://litteraturbanken.se/">Litteraturbanken</a>.
2. Or run this cell that will download the corpus and place it in your folder.

In [2]:
# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname) 
    for fname in 
    [
        "bannlyst.txt", 
        "gosta.txt", 
        "herrgard.txt", 
        "jerusalem.txt", 
        "kejsaren.txt", 
        "marbacka.txt", 
        "nils.txt", 
        "osynliga.txt", 
        "troll.txt"
    ]
]

def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""
    
    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip downloaded.")
        req.close()
        
        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith("__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)
            
        print("Done!")
        
# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")
    
SELMA_FILES

Selma has been downloaded.


['Selma/bannlyst.txt',
 'Selma/gosta.txt',
 'Selma/herrgard.txt',
 'Selma/jerusalem.txt',
 'Selma/kejsaren.txt',
 'Selma/marbacka.txt',
 'Selma/nils.txt',
 'Selma/osynliga.txt',
 'Selma/troll.txt']

### Running the indexer (optional)

In a production context, your final program would take a corpus as input (here the Selma Lagerlöf's novels) and create an index of all the words with their positions. You should be able to run it this way:
<pre>$ python indexer.py folder_name</pre>
In this lab, you will write the index in a Jupyter Notebook. The conversion into a Python program is left as an optional exercise.

## Programming the Indexer

To make programming easier, you will split this exercise into five steps:
1. Index one file;
2. Read the content of a folder
3. Create a master index for all the files
4. Use tfidf to represent the documents (novels)
5. Compare the documents of a collection

You will use dictionaries to represent the postings.

### Indexing one file

#### Description

<p>Write a program that reads one document <tt>file_name.txt</tt> and outputs an index file:
            <tt>file_name.idx</tt>:
        </p>
        <ol>
            <li>The index file will contain all the unique words in the document,
                where each word is associated with the list of its positions in the document.
            </li>
            <li>You will represent this index as a dictionary, where the keys will be the words, and
                the values, the lists of positions
            </li>
            <li>As words, you will consider all the strings of letters that you will set in lower case.
                You will not index the rest (i.e. numbers, punctuations, or symbols).
            </li>
            <li>To extract the words, use Unicode regular expressions. Do not use <tt>\w+</tt>,
                for instance, but the Unicode equivalent.
            </li>
            <li>The word positions will correspond to the number of characters from the beginning of the file.
                (The word offset from the beginning)
            </li>
            <li>You will use the <tt>finditer()</tt> method to find the positions of the words.
                This will return you match objects,
                where you will get the matches and the positions with
                the <tt>group()</tt> and <tt>start()</tt> methods.
            </li>
            <li>You will use the pickle package to write your dictionary in an file,
                see <a href="https://wiki.python.org/moin/UsingPickle">https://wiki.python.org/moin/UsingPickle</a>.
            </li>
        </ol>

Below is an excerpt of the index of the `bannlyst.txt` text for the words <i>gjord</i>, <i>uppklarnande</i>, and <i>stjärnor</i>. The data is stored in a dictionary:

<pre>
{...
'gjord': [8600, 183039, 220445],
'uppklarnande': [8617],
'stjärnor': [8641], ...
}
</pre>
where the word <i>gjord</i> occurs three times in the text at positions 8600, 183039, and 220445, <i>uppklarnande</i>, once at position 8617, and <i>stjärnor</i>, once at position 8641.

#### Writing a tokenizer 

Write a Unicode regular expression to find words defined as sequences of letters.

In [3]:
# Write your regex here
regex = r"\p{L}+"

In [4]:
re.findall(regex, 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa')

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

Using `regex`, write `tokenize(text)` function to tokenize a text. Return their positions.

In [5]:
# Write your code here
def tokenize(text: str):
    return re.finditer(regex, text)

In [6]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')
list(tokens)

[<regex.Match object; span=(0, 2), match='En'>,
 <regex.Match object; span=(3, 7), match='gång'>,
 <regex.Match object; span=(8, 12), match='hade'>,
 <regex.Match object; span=(13, 15), match='de'>,
 <regex.Match object; span=(16, 18), match='på'>,
 <regex.Match object; span=(19, 27), match='Mårbacka'>,
 <regex.Match object; span=(28, 30), match='en'>,
 <regex.Match object; span=(31, 39), match='barnpiga'>,
 <regex.Match object; span=(41, 44), match='som'>,
 <regex.Match object; span=(45, 50), match='hette'>,
 <regex.Match object; span=(51, 55), match='Back'>,
 <regex.Match object; span=(56, 61), match='Kajsa'>]

#### Extracting indices

Write a `text_to_idx(words)` function to extract the indices from the list of tokens (words). Return a dictionary, where the keys will be the tokens (words), and the values a list of positions.

In [7]:
# Write your code here
def text_to_idx(tokens):
    indexing = {}
    for token in tokens:
        word = token.group()
        if word in indexing:
            indexing[word].append(token.start())
        else:
            indexing[word] = [token.start()]
    return indexing


In [8]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.'.lower().strip())
text_to_idx(tokens)

{'en': [0, 28],
 'gång': [3],
 'hade': [8],
 'de': [13],
 'på': [16],
 'mårbacka': [19],
 'barnpiga': [31],
 'som': [41],
 'hette': [45],
 'back': [51],
 'kajsa': [56]}

#### Reading one file

Read one file, _Mårbacka_, `marbacka.txt`, set it in lowercase, tokenize it, and index it. Call this index `idx`

In [9]:
# Write your code here
def read_file(filename):
    return " ".join(open("Selma/" + filename).read().lower().strip().split())
def getIdx(filename):
    file_string = read_file(filename)
    tokens = tokenize(file_string)
    idx = text_to_idx(tokens)
    return idx
idx = getIdx("marbacka.txt")


In [10]:
idx['mårbacka']

[15,
 123,
 735,
 1681,
 2561,
 3303,
 15056,
 15343,
 27680,
 41980,
 48896,
 50173,
 51812,
 59854,
 63064,
 64583,
 66840,
 66988,
 67456,
 67480,
 68884,
 70975,
 71745,
 73788,
 73896,
 74254,
 76241,
 76514,
 76768,
 77138,
 77415,
 77563,
 78192,
 80210,
 81394,
 81614,
 83961,
 84381,
 84844,
 89410,
 96628,
 98165,
 99980,
 104554,
 104788,
 105210,
 108188,
 108610,
 112305,
 113183,
 114452,
 115036,
 115285,
 115818,
 116007,
 121314,
 124223,
 125803,
 125935,
 128139,
 130346,
 131290,
 132176,
 136253,
 136525,
 137208,
 138524,
 140032,
 141624,
 146057,
 150738,
 153566,
 154367,
 154665,
 155111,
 155627,
 157323,
 159025,
 159313,
 160361,
 161279,
 165029,
 167486,
 167697,
 168271,
 169489,
 171834,
 181146,
 181525,
 185451,
 188611,
 190068,
 190178,
 192239,
 195726,
 201564,
 205342,
 206788,
 207378,
 208866,
 209517,
 216430,
 218882,
 220338,
 220477,
 220822,
 221155,
 223126,
 228409,
 228499,
 229683,
 230351,
 231037,
 233315,
 235077,
 235832,
 239039,


#### Saving the index

Save your index in a file so that you can reuse it. Use the pickle module.

In [11]:
# Write your code here
pickle.dump(idx, open("SelmaIndexed/marbacka.idx", "wb"))

Read back your file and store the content in `idx`

In [12]:
# Write your code here
idx_2 = pickle.load(open("SelmaIndexed/marbacka.idx", "rb"))

In [13]:
idx_2['mårbacka']

[15,
 123,
 735,
 1681,
 2561,
 3303,
 15056,
 15343,
 27680,
 41980,
 48896,
 50173,
 51812,
 59854,
 63064,
 64583,
 66840,
 66988,
 67456,
 67480,
 68884,
 70975,
 71745,
 73788,
 73896,
 74254,
 76241,
 76514,
 76768,
 77138,
 77415,
 77563,
 78192,
 80210,
 81394,
 81614,
 83961,
 84381,
 84844,
 89410,
 96628,
 98165,
 99980,
 104554,
 104788,
 105210,
 108188,
 108610,
 112305,
 113183,
 114452,
 115036,
 115285,
 115818,
 116007,
 121314,
 124223,
 125803,
 125935,
 128139,
 130346,
 131290,
 132176,
 136253,
 136525,
 137208,
 138524,
 140032,
 141624,
 146057,
 150738,
 153566,
 154367,
 154665,
 155111,
 155627,
 157323,
 159025,
 159313,
 160361,
 161279,
 165029,
 167486,
 167697,
 168271,
 169489,
 171834,
 181146,
 181525,
 185451,
 188611,
 190068,
 190178,
 192239,
 195726,
 201564,
 205342,
 206788,
 207378,
 208866,
 209517,
 216430,
 218882,
 220338,
 220477,
 220822,
 221155,
 223126,
 228409,
 228499,
 229683,
 230351,
 231037,
 233315,
 235077,
 235832,
 239039,


### Reading the content of a folder

Write a `get_files(dir, suffix)` function that reads all the files in a folder with a specific `suffix` (txt). You will need the Python `os` package, see <a href="https://docs.python.org/3/library/os.html">https://docs.python.org/3/library/os.html</a>. You will return the file names in a list.

You can reuse this function:

In [14]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [15]:
# Write your code here
get_files("Selma", "txt")

['bannlyst.txt',
 'gosta.txt',
 'herrgard.txt',
 'jerusalem.txt',
 'kejsaren.txt',
 'marbacka.txt',
 'nils.txt',
 'osynliga.txt',
 'troll.txt']

### Creating a master index

Complete your program with the creation of master index, where you will associate each word of the corpus with the files, where it occur and its positions: a posting list
Below is an except of the master index with the words <i>samlar</i> and <i>ände</i>:

In [16]:
{'samlar':
            {'troll.txt': [641880, 654233],
            'nils.txt': [51805, 118943],
            'osynliga.txt': [399121],
            'gosta.txt': [313784, 409998, 538165]},
 'ände':
            {'troll.txt': [39562, 650112],
            'kejsaren.txt': [50171],
            'marbacka.txt': [370324],
            'nils.txt': [1794],
            'osynliga.txt': [272144]}
}

{'samlar': {'troll.txt': [641880, 654233],
  'nils.txt': [51805, 118943],
  'osynliga.txt': [399121],
  'gosta.txt': [313784, 409998, 538165]},
 'ände': {'troll.txt': [39562, 650112],
  'kejsaren.txt': [50171],
  'marbacka.txt': [370324],
  'nils.txt': [1794],
  'osynliga.txt': [272144]}}

The word <i>samlar</i>, for instance, occurs three times in the gosta text at positions
            313784, 409998, and 538165.

In [17]:
# write your code here
index_list = map((lambda filename: [filename, getIdx(filename)]), get_files("Selma", "txt"))
master_index = {}
for [filename, index] in index_list:
    for word in index.keys():
        if word in master_index.keys():
            master_index[word][filename] = index[word]
        else:
            master_index[word] = {}
            master_index[word][filename] = index[word]

            
        

In [18]:
master_index['samlar']

{'gosta.txt': [311889, 407517, 535099],
 'nils.txt': [51388, 118292],
 'osynliga.txt': [397432],
 'troll.txt': [638950, 651238]}

In [19]:
master_index['mårbacka']

{'marbacka.txt': [15,
  123,
  735,
  1681,
  2561,
  3303,
  15056,
  15343,
  27680,
  41980,
  48896,
  50173,
  51812,
  59854,
  63064,
  64583,
  66840,
  66988,
  67456,
  67480,
  68884,
  70975,
  71745,
  73788,
  73896,
  74254,
  76241,
  76514,
  76768,
  77138,
  77415,
  77563,
  78192,
  80210,
  81394,
  81614,
  83961,
  84381,
  84844,
  89410,
  96628,
  98165,
  99980,
  104554,
  104788,
  105210,
  108188,
  108610,
  112305,
  113183,
  114452,
  115036,
  115285,
  115818,
  116007,
  121314,
  124223,
  125803,
  125935,
  128139,
  130346,
  131290,
  132176,
  136253,
  136525,
  137208,
  138524,
  140032,
  141624,
  146057,
  150738,
  153566,
  154367,
  154665,
  155111,
  155627,
  157323,
  159025,
  159313,
  160361,
  161279,
  165029,
  167486,
  167697,
  168271,
  169489,
  171834,
  181146,
  181525,
  185451,
  188611,
  190068,
  190178,
  192239,
  195726,
  201564,
  205342,
  206788,
  207378,
  208866,
  209517,
  216430,
  218882,
  22033

Save your master index in a file and read it again

In [20]:
# Write your code here
pickle.dump(master_index, open("master_index.idx", "wb"))
master_index = pickle.load(open("master_index.idx", "rb"))

In [21]:
master_index['samlar']

{'gosta.txt': [311889, 407517, 535099],
 'nils.txt': [51388, 118292],
 'osynliga.txt': [397432],
 'troll.txt': [638950, 651238]}

#### Concordances

Write a `concordance(word, master_index, window)` function to extract the concordances of a `word` within a window of `window` characters

In [22]:
# Write your code here
def concordance(word, master_index, window):
    result = {}
    for file in master_index[word].keys():
        file_string = read_file(file)
        result[file] = []
        for start in master_index[word][file]:
            result[file].append(file_string[max(start-window, 0):start+window])
    return result



In [23]:
concordance('samlar', master_index, 25)

{'gosta.txt': ['om ligger nära borg, och samlar ihop ett litet mid',
  ' lika förstämda. men hon samlar upp allt detta som',
  'an ensam i livet. därmed samlar han korten tillhop'],
 'nils.txt': [' bara, att du i all hast samlar ihop så mycket bos',
  'ar stannat hemma, och nu samlar de sig för att int'],
 'osynliga.txt': [' till höger i kärran och samlar just ihop tömmarna'],
 'troll.txt': ['en örtkunnig läkare, som samlar in markens växter ',
  'älper dem, och medan hon samlar och handlar för de']}

### Representing Documents with tf-idf

Once you have created the index, you will represent each document in your corpus as a dictionary. The keys of these dictionaries will be the words and you will define the value of a word with the tf-idf metric: 
1. Read the description of the tf-idf measure on Wikipedia (<a href="https://en.wikipedia.org/wiki/Tf%E2%80%93idf">https://en.wikipedia.org/wiki/Tf-idf</a>)
2. After reading the description, you probably realized that there are multiple definitions of tf-idf. In this assignment, 
 * Tf will be the relative frequency of the term in the document and 
 * idf, the logarithm base 10 of the inverse document frequency.
        
You have below the tf-idf values for a few words. In our example, the word <i>gås</i> has the value 0 in bannlyst.txt and the value 0.000101001964 in nils.txt

<pre>
troll.txt
	känna	 0.0
	gås	 0.0
	nils	 2.148161748868631e-06
	et	 0.0
kejsaren.txt
	känna	 0.0
	gås	 0.0
	nils	 8.08284798629935e-06
	et	 8.273225429362848e-05
marbacka.txt
	känna	 0.0
	gås	 0.0
	nils	 7.582276564686669e-06
	et	 9.70107989686256e-06
herrgard.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
nils.txt
	känna	 0.0
	gås	 0.00010100196417506702
	nils	 0.00010164426900380124
	et	 0.0
osynliga.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
jerusalem.txt
	känna	 0.0
	gås	 0.0
	nils	 4.968292117670952e-06
	et	 0.0
bannlyst.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
gosta.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
</pre>

Conceptually, the tf-idf representation is a vector. In your program, you will keep this idea and use all the words in the corpus as keys: Each dictionary will include all the words of the corpus as keys. The value of the key is then possibly 0, meaning that the word in not in the document or is in all the documents as for the word `nils` in `gosta.tx`. 

As further work, you may think of optimizing this part.

In [24]:
# Write your code here
from collections import defaultdict
selma_files = get_files("Selma", "txt")
words_in_document = {}
for filename in selma_files:
    file_string = read_file(filename)
    words_in_document[filename] = len(list(tokenize(file_string)))

tfidf = defaultdict(lambda: defaultdict(lambda: 0))
for word in master_index.keys():
    for file_name in selma_files:
        tf = 0
        idf = 0
        if file_name in master_index[word]:
            times = len(master_index[word][file_name])
            tf = times / words_in_document[file_name]
            idf = math.log10(len(selma_files) / len(master_index[word]))
        tfidf[file_name][word] = tf * idf

In [25]:
tfidf['troll.txt']['känna']

0.0

In [26]:
tfidf['troll.txt']['nils']

2.1481617488686316e-06

### Comparing Documents

Using the cosine similarity, compare all the pairs of documents with their tf-idf representation and present your results in a table. You will include this table in your report.

#### Cosine similarity

Write a function computing the cosine similarity between two documents: `cosine_similarity(document1, document2)`

In [27]:
# Write your code here
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(file_name_1, file_name_2):
    words = master_index.keys()
    tfidf_1 = []
    tfidf_2 = []
    for word in words:
        tfidf_1.append(tfidf[file_name_1][word])
        tfidf_2.append(tfidf[file_name_2][word])
    return dot(tfidf_1, tfidf_2) / (norm(tfidf_1) * norm(tfidf_2))

print(selma_files[1])
print(selma_files[2])
cosine_similarity(selma_files[1], selma_files[2])

gosta.txt
herrgard.txt


0.0031103994916002096

#### Similarity matrix

Compute the similarity matrix between the documents of the corpus. While computing the similarities, you will record the two most similar documents that you will call `most_sim_doc1` and `most_sim_doc2`.

In [28]:
import numpy as np
# Write your code here
matrix = [["matrix", *selma_files]]
for file_1 in selma_files:
    row = [file_1]
    for file_2 in selma_files:
        row.append(round(cosine_similarity(file_1, file_2), 4))
    matrix.append(row)

s = [[str(e) for e in row] for row in matrix]
lens = [max(map(len, col)) for col in zip(*s)]
fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
table = [fmt.format(*row) for row in s]
print('\n'.join(table))

most_sim_doc1 = ""
most_sim_doc2 = ""
max_similarity = 0
for i in range(0, len(matrix)):
    for j in range (0, len(matrix[i])):
        if i == j or i == 0 or j == 0:
            next
        elif matrix[i][j] > max_similarity:
            max_similarity = matrix[i][j]
            most_sim_doc1 = matrix[i][0]
            most_sim_doc2 = matrix[0][j]
        

        

matrix       	bannlyst.txt	gosta.txt	herrgard.txt	jerusalem.txt	kejsaren.txt	marbacka.txt	nils.txt	osynliga.txt	troll.txt
bannlyst.txt 	1.0         	0.049    	0.0009      	0.0065       	0.024       	0.0368      	0.051   	0.0521      	0.0886   
gosta.txt    	0.049       	1.0      	0.0031      	0.0043       	0.048       	0.0802      	0.1048  	0.1248      	0.1957   
herrgard.txt 	0.0009      	0.0031   	1.0         	0.3707       	0.0007      	0.0036      	0.0051  	0.0048      	0.0041   
jerusalem.txt	0.0065      	0.0043   	0.3707      	1.0          	0.0018      	0.0049      	0.0045  	0.0283      	0.0071   
kejsaren.txt 	0.024       	0.048    	0.0007      	0.0018       	1.0         	0.0711      	0.0497  	0.0511      	0.1813   
marbacka.txt 	0.0368      	0.0802   	0.0036      	0.0049       	0.0711      	1.0         	0.0847  	0.0932      	0.1472   
nils.txt     	0.051       	0.1048   	0.0051      	0.0045       	0.0497      	0.0847      	1.0     	0.1106      	0.1885   
osynliga.txt 	0.0521    

Give the name of the two novels that are the most similar.

In [29]:
print("Most similar:", most_sim_doc1, most_sim_doc2, "Similarity:", max_similarity)

Most similar: herrgard.txt jerusalem.txt Similarity: 0.3707


## Submission

When you have written all the code and run all the cells, show it to one of the assistants who will pass you.

In case, we do not have enough assitants, you will submit it to an automatic system. You will have more information later on this if we need it. In this case, fill in your IDs and as well as the name of the notebook.

In [30]:
STIL_ID = ["student1", "student2"] # Write your stil ids
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "1-indexer_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the two most similar novels.

In [31]:
ANSWER = ' '.join(sorted([most_sim_doc1, most_sim_doc2]))
ANSWER

'herrgard.txt jerusalem.txt'

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [32]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [33]:
ASSIGNMENT = 1
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/jonat/progProject/Plugg/sprak-tek/edan20/labs_2023/1-indexer_solution.ipynb'

In [ ]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=False)


# from IPython.display import display, JSON
res.json()

/Users/pierre/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vilde.cs.lth.se'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'msg': None,
 'status': 'correct',
 'signature': '62404b10eabf9afe5db5c548ed802ff0e102292b0da3e65ef9c9f95984c59b7e91d9ba31fe3d3359581c4ea2ec7fc4e10c243167d260681376d04ecd4f76f92c',
 'submission_id': '222462ad-305f-47d7-9ee8-2d1c5ffd9181'}

Check the `status` and be sure it is `correct`. If not, revise your code; verify that you obtained intermediate results identical to those in the notebook; and resubmit your notebook. You can submit multiple times.

<h2>Turning in your assigment</h2>

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program, 
2. Read the text <i>Challenges in Building Large-Scale Information Retrieval Systems</i> about the history of <a href="https://research.google.com/people/jeff/WSDM09-keynote.pdf">Google indexing</a> by <a href="https://research.google.com/pubs/jeff.html">Jeff Dean</a> and write a short comment on it. See the details below.

You will submit your report as well as your notebook (for archiving purposes) to Canvas: <https://canvas.education.lu.se/>. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (<a href="https://www.overleaf.com/">www.overleaf.com</a>). This will probably help you structure your text. You will then upload a PDF file in Canvas.

In your report of about two pages: 
1. You will describe your indexer and comment your results; in this description, you will write the regular expression you used for the tokenization and include the similarity matrix;
2. In Jeff Dean's document, you will identify the slide, where you have the most similar indexing technique. Please write the slide title and the slide number in your report.
3. You will tell how your index encoding is related to what Google did.


The submission deadline is September 15, 2023. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.